In [8]:
import json
import os
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document

# Charger la clé API
load_dotenv()

# Charger ta base de données JSON
with open("operations_administratives_maroc.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Transformer les données en Documents LangChain
documents = []
for category, docs in data.items():
    for doc in docs:
        text = f"""
ID: {doc['id']}
Opération: {doc['operation']}
Catégorie: {doc['categorie']}
Documents requis: {', '.join(doc['documents_requis'])}
Nombre de documents: {doc['nombre_documents']}
Autorité compétente: {doc['autorite_competente']}
Source officielle: {doc['source_officielle']}
Frais estimés: {doc['frais_estimés']}
Délai moyen: {doc['delai_moyen']}
Format de sortie: {doc['format_sortie']}
"""
        documents.append(Document(page_content=text, metadata={"id": doc['id'], "categorie": doc['categorie']}))

# Initialiser le modèle d'embeddings et créer l'index vectoriel
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(documents, embeddings)

# Créer la chaîne de question/réponse avec récupération par similarité
qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-4o-mini", temperature=0),
    retriever=vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3}),
    return_source_documents=True
)


🟢 Chat administratif intelligent prêt !
Pose ta question (ex: Comment renouveler ma CNIE ?)
Tape 'exit' pour quitter.



❓ Ta question :  comment je peux faire mon passport ?



💬 Réponse :
Pour faire votre passeport biométrique pour la première demande, vous devez suivre ces étapes :

1. Rassembler les documents requis :
   - Formulaire de demande rempli
   - Copie de votre Carte Nationale d'Identité Électronique (CNIE)
   - 4 photos d'identité récentes
   - Acte de naissance ou livret de famille
   - Code e-timbre (timbre électronique)

2. Soumettre votre demande auprès de l'autorité compétente, qui peut être la DGSN, la Préfecture ou le Consulat.

3. Payer les frais estimés de 300 DH.

4. Attendre le délai moyen de traitement, qui est de 1 à 3 semaines.

Pour plus d'informations, vous pouvez consulter le site officiel : [passeport.ma](https://www.passeport.ma).

📄 Sources :
- ID: 4, Catégorie: Documents d'identité
- ID: 5, Catégorie: Documents d'identité
- ID: 6, Catégorie: Documents d'identité




❓ Ta question :  exit


👋 Fin de la session. À bientôt !


In [9]:
# === Boucle interactive ===
print("\n🟢 Chat administratif intelligent prêt !")
print("Pose ta question (ex: Comment renouveler ma CNIE ?)")
print("Tape 'exit' pour quitter.\n")

while True:
    question = input("❓ Ta question : ")
    if question.lower() == "exit":
        print("👋 Fin de la session. À bientôt !")
        break

    result = qa(question)
    print("\n💬 Réponse :")
    print(result['result'])


🟢 Chat administratif intelligent prêt !
Pose ta question (ex: Comment renouveler ma CNIE ?)
Tape 'exit' pour quitter.



❓ Ta question :  exit


👋 Fin de la session. À bientôt !
